### references
- aspect based sentiment analysis: https://github.com/ScalaConsultants/Aspect-Based-Sentiment-Analysis

### import packages

In [1]:
from utils import data_scraping, absa_english_text
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

2023-09-06 16:39:46.951638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-06 16:39:46.951749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/tdjiang/.local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-06 16:39:51.283687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-09-06 16:39:51.283741: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-09-06 16

### read & transform data

In [2]:
# read data
file_name = 'data_the_grace_dalat_reviews.csv'
if os.path.exists(file_name):
    df = pd.read_csv(file_name, converters={'profileId':str})
    eval_cols = ['travelPurpose','travelKeywords','photoDataDisplaysList','reactionSummaries']
    df[eval_cols] = df[eval_cols].applymap(lambda x: eval(x) if isinstance(x, str) else x)
else:
    data = data_scraping(url_hotel="https://www.traveloka.com/vi-vn/hotel/vietnam/the-grace-hotel-dalat-3000010042556", reviews_per_page=10)
    df = data.get_all_reviews()
    df.to_csv(file_name, index=False)

In [3]:
# replace empty data to blank data
df = df.mask(df == '')

# drop columns with no data
df = df.dropna(axis=1, how='all')

# add new columns
df['travelPurposeText'] = df.travelPurpose.apply(lambda x: x['travelPurposeText'] if isinstance(x, dict) else x)
df['travelPurpose'] = df.travelPurpose.apply(lambda x: x['travelPurpose'] if isinstance(x, dict) else x)
df['travelKeywords'] = df.travelKeywords.apply(lambda x: ','.join(sorted(map(lambda y: y['travelKeyword'] if isinstance(y, dict) else '', x))) if isinstance(x, list) else x)
df['reviewLikes'] = df.reactionSummaries.apply(lambda x: x['reactionSummaryMap']['LIKE']['reactionCount'] if isinstance(x, dict) else x)
df['photoCategories'] = df.photoDataDisplaysList.apply(lambda x: ','.join(sorted(map(lambda y: y['photoCategoryDisplay']['photoCategory'] if isinstance(y, dict) else '', x))) if isinstance(x, list) else x)

In [4]:
output = []

file_name = 'output_the_grace_dalat_reviews.csv'
if os.path.exists(file_name):
    df_output_fn = pd.read_csv(file_name)
    finishedReviewId = df_output_fn.reviewId.unique().tolist()
else:
    df_output_fn = pd.DataFrame(columns=['text','aspect','sentiment','neu_score','neg_score','pos_score','reviewId'])
    finishedReviewId = []

for id, row in enumerate(df[~df.reviewId.isin(finishedReviewId)].to_dict(orient='records')[:15]):
    reviewTextFn = ''
    absa_class = absa_english_text(row['originalReviewText'])
    if row['translated']:
        reviewTextFn = row['reviewText']
    else:
        tokenized = absa_class.words_tokenized()
        reviewTextFn = absa_class.translate_vi_to_en(tokenized)
    
    sentiments = absa_class.absa_by_np(reviewTextFn)
    output.append({
        'reviewId': row['reviewId'],
        'sentiment':sentiments
    })
    print('---', id)

--- 0
--- 1


: 

: 

In [6]:
df_output = pd.DataFrame(output)
df_output['sentiment'] = df_output.apply(lambda x: pd.DataFrame(x.sentiment).assign(reviewId = x.reviewId) , axis=1)
df_output = pd.concat(df_output.sentiment.tolist())
pd.concat([df_output_fn, df_output]).to_csv('output_the_grace_dalat_reviews.csv', index=False)